In [27]:
from collections import defaultdict
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import calendar
from scipy.optimize import curve_fit


In [7]:
#estimator I built from previous part 

class Estimator:

    def __init__(self,start_range,end_range,dateframe):
        self.start_range = start_range
        self.end_range = end_range
        self.df = dateframe


    def convert_dates_to_year_fraction(self,dates):
        start = self.df.index[0]
        return np.array([(d - start).days / 365 for d in dates])



    def ml_estimate(self, date):

        dates = self.df.index


        y = self.df.loc[dates].squeeze().values
        t = np.array(self.convert_dates_to_year_fraction(dates))

        def seasonal_model(t, a, phi, b, c):
            return a * np.sin(2 * np.pi * t + phi) + b * t + c

        guess = [np.std(y), 0, 0, np.mean(y)]

        popt, _ = curve_fit(seasonal_model, t, y, p0=guess)

        t_future = (date - dates.min()).days / 365
        prediction = seasonal_model(t_future, *popt)
        return prediction





    def simple_estimate(self,date):
        if date > self.start_range and date < self.end_range:
            return self.estimate_past(date) 
        else:
            return self.predict_future(date)
    

    #this linearly interpolates from the prices of the two closest dates 
    def estimate_past(self,date):
        
        
        prev,next = self.get_surrounding_dates(date)
        price_prev = self.df.loc[prev].item()
        price_next = self.df.loc[next].item()

        #print(price_prev,price_next)


        total_span = (next - prev).total_seconds()
        elapsed = (date - prev).total_seconds()
        
        weight = elapsed / total_span
        interpolated = price_prev + weight * (price_next - price_prev)


        return interpolated
    

    #this takes the final two dates and extrapolates outward assuming constant slope
    def predict_future(self, date):

        last_two_dates = self.df.index[self.df.index <= self.end_range].sort_values()[-2:]
        
        t1, t2 = last_two_dates[0], last_two_dates[1]
        y1 = self.df.loc[t1].item()
        y2 = self.df.loc[t2].item()

        delta_t = (t2 - t1).total_seconds()
        slope = (y2 - y1) / delta_t

        delta_future = (date - t2).total_seconds()
        prediction = y2 + slope * delta_future

        return prediction



    
    def get_surrounding_dates(self,date):
        prev_month = date.replace(day=1) - timedelta(days=1)
        prev_eom = prev_month.replace(day=calendar.monthrange(prev_month.year, prev_month.month)[1])
    
        this_eom = date.replace(day=calendar.monthrange(date.year, date.month)[1])
    
        return (prev_eom, this_eom)




In [39]:
class GasStoragePricer:
    def __init__(self,estimator,inj_rate,with_rate,max_volume,storage_cost_per_month,inj_with_cost_per_mmbtu):
        self.estimator = estimator
        self.inj_rate = inj_rate
        self.with_rate = with_rate
        self.max_volume = max_volume
        self.storage_cost_per_month = storage_cost_per_month
        self.inj_with_cost_per_mmbtu = inj_with_cost_per_mmbtu

    def price_contract(self, injection_dates, withdrawal_dates):
        volume = 0
        cash_flows = defaultdict(float)

        all_dates = sorted(set(injection_dates + withdrawal_dates))

        for date in all_dates:
            price = self.estimator.ml_estimate(date)

            if date in injection_dates:
                inject_volume = min(self.inj_rate, self.max_volume - volume)
                volume += inject_volume
                cost = price * inject_volume + self.inj_with_cost_per_mmbtu * inject_volume
                cash_flows[date] -= cost

            if date in withdrawal_dates:
                withdraw_volume = min(self.with_rate, volume)
                volume -= withdraw_volume
                revenue = price * withdraw_volume - self.inj_with_cost_per_mmbtu * withdraw_volume
                cash_flows[date] += revenue

        months = (max(withdrawal_dates).year - min(injection_dates).year) * 12 + \
                 (max(withdrawal_dates).month - min(injection_dates).month) + 1
        
        total_storage_cost = months * self.storage_cost_per_month

        net_value = sum(cash_flows.values()) - total_storage_cost

        return net_value
        


# Using the model

In [40]:
df = pd.read_csv('Nat_Gas.csv')
df['Dates'] = pd.to_datetime(df['Dates'], format='%m/%d/%y')
df.set_index('Dates',inplace=True)
obj = Estimator(df.index[0],df.index[-1],df)

In [41]:
pricer = GasStoragePricer(
    estimator=obj,
    inj_rate=500000,
    with_rate=500000,
    max_volume=1e6,
    storage_cost_per_month=100000,
    inj_with_cost_per_mmbtu=0.01
)

injection_dates= [datetime(2025, 6, 30), datetime(2025, 7, 31)]
withdrawal_dates= [datetime(2025, 12, 31), datetime(2026, 1, 31)]

contract_val = pricer.price_contract(injection_dates,withdrawal_dates)
print(contract_val)





729536.0108330492
